In [18]:
import tensorflow as tf

In [1]:
from keras.applications import MobileNet

#we are transforming already created model

Using TensorFlow backend.


In [2]:
model = MobileNet(weights = 'imagenet', input_shape = (224,224,3), include_top = False)

#here we set our own input_shape
#to make the output layer of softmax off so that we can add our own layers at the end

17227776/17225924 [==============================] - 4s 0us/step


In [3]:
model.layers

In [4]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [5]:
model.layers[0].__class__.__name__

#To get the name of a layer

'InputLayer'

In [6]:
model.layers[1].__class__.__name__

'ZeroPadding2D'

In [8]:
model.layers[0].trainable

#to check if the layer is trainable or not

False

In [12]:
print("Name of layer : ", model.layers[1].__class__.__name__ )
print("Is is trainable or not :", model.layers[1].trainable )

Name of layer :  ZeroPadding2D
Is is trainable or not : True


In [13]:
#Now we have to freeze all the layers 
#We already disable the output layer above

In [14]:
#To freeze all the layers

for layer in model.layers:
    layer.trainable = False

In [15]:
model.output #we disable the softmax outer layer

<tf.Tensor 'conv_pw_13_relu/Relu6:0' shape=(None, 7, 7, 1024) dtype=float32>

In [16]:
top_model = model.output

#We store this output layer

In [17]:
top_model

<tf.Tensor 'conv_pw_13_relu/Relu6:0' shape=(None, 7, 7, 1024) dtype=float32>

In [38]:
#Adding layers to the end 

from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model

top_model = Dense(1024, activation= 'relu')(top_model)

In [25]:
top_model = Dense(512, activation= 'relu')(top_model)

In [26]:
top_model = Dense(10, activation= 'softmax')(top_model)
#this is our own output model

In [29]:
print("Our Previous Model that is frozen = " , model.input)
print("Our New Added Model = ", top_model)

Our Previous Model that is frozen =  Tensor("input_1:0", shape=(None, 224, 224, 3), dtype=float32)
Our New Added Model =  Tensor("dense_4/truediv:0", shape=(None, 7, 7, 10), dtype=float32)


In [39]:
#Combining both to create new model

new_model = Model(inputs = model.input, outputs = top_model)

In [41]:
new_model.summary()

#our 3 new layers are added

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [ ]:
#Now whenever we fit/train our model only our 3 new layers will get trained 
#other layer will remain intact because we already freeze them